In [ ]:
!pip install transformers torch sentencepiece pandas scikit-learn warcio boto3 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 833.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [10]:
import urllib.request
import zipfile
import os

# URL del archivo ZIP del corpus deseado
url = "https://object.pouta.csc.fi/OPUS-Books/v1/moses/en-es.txt.zip"
file_name = "Books-en-es.zip"

# Descarga el archivo
urllib.request.urlretrieve(url, file_name)

# Descomprimir el archivo
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("Books-en-es")

# Listar archivos descomprimidos
os.listdir("Books-en-es")


['Books.en-es.en', 'Books.en-es.es', 'README', 'Books.en-es.ids']

In [11]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
# Texto en español
text = "Pienso que la Inteligencia Artificial no va a quitarnos nuestros trabajos, los va a potencializar"

inputs = tokenizer(text, return_tensors="pt", padding=True)
translated_tokens = model.generate(**inputs)
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Texto traducido al inglés
print(translated_text)

I think Artificial Intelligence is not going to take our jobs away from us, it's going to enhance them.


In [13]:
!pip install transformers torch google-cloud-translate

In [15]:
import pandas as pd

# Definir el dataset como un DataFrame con algunos ejemplos
data = {
    "en": [
        "Check out the latest summer collection",
        "Enjoy huge discounts on Black Friday",
        "Unlock the full potential of your smartphone with our new range of accessories"
    ],
    "es_es": [
        "Descubre la última colección de verano",
        "Disfruta de enormes descuentos este Viernes Negro",
        "Desbloquea todo el potencial de tu smartphone con nuestra nueva gama de accesorios"
    ],
    "es_co": [
        "Mira la nueva colección de verano",
        "No te puedes perder estos descuentazos del Viernes Negro",
        "Aprovecha al máximo tu celular con nuestra nueva línea de accesorios"
    ]
}
df = pd.DataFrame(data)


In [16]:
#Crear un Modelo de Traducción y Adaptación Cultural
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-es'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [17]:
# Ajustar las traducciones estándar a variantes locales utilizando reglas específicas

def adapt_translation(translation, country="es"):
    if country == "es":
        return translation.replace("celular", "móvil").replace("descuentazos", "descuentos")
    elif country == "co":
        return translation.replace("móvil", "celular").replace("descuentos", "descuentazos")
    else:
        return translation

# Ejemplo de uso
text_en = "Check out the latest summer collection"
translation = translate(text_en)
print("Standard Spanish:", translation)
print("Spanish for Spain:", adapt_translation(translation, "es"))
print("Spanish for Colombia:", adapt_translation(translation, "co"))


Standard Spanish: Echa un vistazo a la última colección de verano
Spanish for Spain: Echa un vistazo a la última colección de verano
Spanish for Colombia: Echa un vistazo a la última colección de verano
